In [10]:
%run -i met.ipynb
get_met()

print(df)

Data collected from frost.met.no!
                       elementId  value  unit  \
0      mean(air_temperature P1D)    4.2  degC   
1      mean(air_temperature P1D)    2.5  degC   
2  sum(precipitation_amount P1D)    0.0    mm   
3  sum(precipitation_amount P1D)    0.0    mm   
4           mean(wind_speed P1D)    2.8   m/s   

                                               level timeOffset  \
0  {'levelType': 'height_above_ground', 'unit': '...       PT0H   
1  {'levelType': 'height_above_ground', 'unit': '...       PT6H   
2                                                NaN      PT18H   
3                                                NaN       PT6H   
4  {'levelType': 'height_above_ground', 'unit': '...       PT0H   

  timeResolution  timeSeriesId performanceCategory exposureCategory  \
0            P1D             0                   C                2   
1            P1D             0                   C                2   
2            P1D             0                   C     

,elementId,value,unit,level,timeOffset,timeResolution,timeSeriesId,performanceCategory,exposureCategory,qualityCode,referenceTime,sourceId
0,mean(air_temperature P1D),4.2,degC,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,0.0,2024-03-18T00:00:00.000Z,SN68860:0
1,mean(air_temperature P1D),2.5,degC,"{'levelType': 'height_above_ground', 'unit': '...",PT6H,P1D,0,C,2,NaN,2024-03-18T00:00:00.000Z,SN68860:0
2,sum(precipitation_amount P1D),0.0,mm,NaN,PT18H,P1D,0,C,2,NaN,2024-03-18T00:00:00.000Z,SN68860:0
3,sum(precipitation_amount P1D),0.0,mm,NaN,PT6H,P1D,0,C,2,0.0,2024-03-18T00:00:00.000Z,SN68860:0
4,mean(wind_speed P1D),2.8,m/s,"{'levelType': 'height_above_ground', 'unit': '...",PT0H,P1D,0,C,2,2.0,2024-03-18T00:00:00.000Z,SN68860:0


()
